In [1]:
import torch
import time
import pandas as pd 
import numpy as np
import regex as re
import warnings
warnings.filterwarnings('ignore')
from torch import nn
from tqdm import tqdm
from rouge import Rouge
from torch.utils.data import Dataset, DataLoader
from googletrans import Translator
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, T5ForConditionalGeneration
from sklearn.model_selection import train_test_split

device = "cuda" if torch.cuda.is_available()  else "cpu"
device

2024-11-20 15:35:28.794516: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732113328.813651   11899 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732113328.819701   11899 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-20 15:35:28.842967: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


'cuda'

## **GRUPO A**
* JAIME ÁLVAREZ URUEÑA
* ALEJANDRO MENDOZA MEDINA

# Explicación de la práctica

El reto sobre el que se va a trabajar en esta práctica es la tarea de detoxificación de frases en distintos idiomas. <br>
Como las oraciones pueden estar hasta en 9 idiomas distintos, la aproximación elegida será primero traducir todas las oraciones a un idioma (inglés),
y entrenar el detoxificador únicamente en inglés. <br>
El pipeline de inferencia será por tanto traducir la frase del idioma original al inglés, detoxificar la frase con el detoxificador entrenado, y volver a traducir
la frase al idioma original.
Con respecto a los Datasets utilizados, se han utilizado 2. El dataset propio de la tarea que cuenta con 400 frases por cada idioma (9), y también se ha utilizado un dataset en el que únicamente tiene ejemplos en inglés. <br>
El modelo de NLP usado es el modelo T5 flan-base. Se ha modificado sus settings para que la tarea que desempeñe sea detoxificar.<br>

Para traducir se han explorado 3 aproximaciones:
- Modelo NLLB-200-distilled-600M, que tiene todos los idiomas de la tarea. No obstante, no tokeniza todos los emojis <br>
- NLLB-200-distilled-600M separando las oraciones por emojis, traduciendo cada porción de la frase original y volviendo a unir por emojis <br>
- Google translate, que contiene todos los idiomas necesarios y sí acepta todos los emojis <br>
Se han implementado las 3 aproximaciones, y la que mejor resultados ofrece es Google Translate, por eso se elige finalmente ese método. <br>
<br>
 


# Translate all phrases (ALREADY DONE)

## Translate with Google Translate (accepts emojis)

In [ ]:
splits = {'eng_Latn': 'data/en-00000-of-00001.parquet', 
          'rus_Cyrl': 'data/ru-00000-of-00001.parquet', 
          'ukr_Cyrl': 'data/uk-00000-of-00001.parquet', 
          'deu_Latn': 'data/de-00000-of-00001.parquet', 
          'spa_Latn': 'data/es-00000-of-00001.parquet', 
          'amh_Ethi': 'data/am-00000-of-00001.parquet', 
          'zho_Hans': 'data/zh-00000-of-00001.parquet', 
          'arb_Arab': 'data/ar-00000-of-00001.parquet', 
          'hin_Deva': 'data/hi-00000-of-00001.parquet'}
translator = Translator()
all_data=[]
for key, value in splits.items():
    x=0
    final_dataframe = pd.DataFrame(columns=['toxic_sentence_not_translated','neutral_sentence_not_translated','toxic_sentence_translated','neutral_sentence_translated','original_language'])
    df = pd.read_parquet("hf://datasets/textdetox/multilingual_paradetox/" + value)
    array = np.array(df)
    for instance in array:
        input=[]
        input.append(instance[0])
        input.append(instance[1])
        toxic_translated = translator.translate(instance[0]).text
        input.append(toxic_translated)
        non_toxic_translated = translator.translate(instance[1]).text
        input.append(non_toxic_translated)
        input.append(key)
        final_dataframe.loc[x]=input
        x+=1
    all_data.append(final_dataframe)
big_dataframe=pd.concat(all_data)
big_dataframe.to_csv('Dataset_tab.csv',sep='\t',index=False)



## Translate with NLLB-200-distilled-600M (does not accept all emojis)

In [ ]:
#con emojis
class Translator_emojis():
    def __init__(self):
        # Use the original NLLB tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M")
        self.model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M")

    def translate(self, src_lan, tgt_lan, text):
        # Split text into words and emojis to handle emojis separately
        words = re.split(r'(\p{So}|\p{Cn})', text, flags=re.UNICODE)
        translated_text = []
        for word in words:
            if word and not re.search(r'[\U0001F600-\U0001F64F\U0001F300-\U0001F5FF\U0001F680-\U0001F6FF]', word):
                # Only translate text segments, skip emojis
                text_with_lang_code = f"{src_lan} {word}"
                inputs = self.tokenizer(text_with_lang_code, return_tensors="pt")
                translated_tokens = self.model.generate(
                    input_ids=inputs["input_ids"],
                    attention_mask=inputs["attention_mask"],
                    forced_bos_token_id=self.tokenizer.convert_tokens_to_ids(tgt_lan)
                )
                translated_word = self.tokenizer.decode(translated_tokens[0], skip_special_tokens=True)
            else:
                # If it's an emoji, keep it as is
                print(u'word')
                translated_text += u'\U0001f604'

            # Append translated word or emoji to result
            translated_text.append(translated_word)
        
        return translated_text
    
#Omitiendo emojis
class Translator():
    def __init__(self):
        self.tokenizer=AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M")
        self.model=AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M")
    def translate(self,src_lan,tgt_lan,text):
        text=f"{src_lan} {text}"
        inputs = self.tokenizer(text, return_tensors="pt")
        translated_tokens = self.model.generate(** inputs, forced_bos_token_id=self.tokenizer.convert_tokens_to_ids(tgt_lan))
        translated=self.tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]
        return translated

In [ ]:
translator=Translator()
translator_emojis = Translator_emojis()
print(translator.translate(src_lan='spa_Latn',tgt_lan='eng_Latn',text="Example text with emoji 🤯 and symbol ♡."))
print(translator_emojis.translate(src_lan='spa_Latn',tgt_lan='eng_Latn',text="Example text with emoji 🤯 and symbol ♡."))

Example text with emoji  and symbol ♡.
['Example text with emoji ', '', 'and symbol', '♪ ♪ ♪', "- I 'm not ."]


# Mergin Datasets (ALREADY DONE)

In [ ]:
languages = pd.read_csv('Dataset_tab.csv',sep='\t')

english = pd.read_csv('train.csv',sep='\t')
english['toxic_sentence_translated']=np.array(english['en_toxic_comment'])
english['neutral_sentence_translated']=np.array(english['en_neutral_comment'])
lang = ["eng_Latn" for i in range(len(english))]
english['original_language']=lang
english = english.rename(columns={'en_toxic_comment': 'toxic_sentence_not_translated', 'en_neutral_comment': 'neutral_sentence_not_translated'})
english.to_csv('english_lines_translated.csv',sep='\t',index=False)

In [ ]:
total_dataset = pd.concat([languages,english])
total_dataset.to_csv('total_dataset_translated.csv',sep='\t',index=False)

# Functionalities

In [2]:
def generate(model,inputs):
    tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
    inputs = " ".join(inputs.split())
    input_ids = tokenizer(inputs, return_tensors="pt").input_ids
    input_ids = input_ids.to(device)
    output_ids = model.generate(input_ids=input_ids, max_length=50, num_beams=5, early_stopping=True)
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return output_text

def compute_rouge(hypothesis,reference):
    rouge = Rouge()
    scores = rouge.get_scores(hypothesis, reference)
    return scores

def detox(model,input,language):
    language_mapping = {
        'eng_Latn': 'en',  # English
        'rus_Cyrl': 'ru',  # Russian
        'ukr_Cyrl': 'uk',  # Ukrainian
        'deu_Latn': 'de',  # German
        'spa_Latn': 'es',  # Spanish
        'amh_Ethi': 'am',  # Amharic
        'zho_Hans': 'zh-cn',  # Simplified Chinese
        'arb_Arab': 'ar',  # Arabic
        'hin_Deva': 'hi'   # Hindi
    }
    language_short=language_mapping[language]
    translator=Translator()
    translated=translator.translate(input,src=language_short,dest='en').text
    detox=generate(model,translated)
    original_language_detox = translator.translate(detox,src='en',dest=language_short).text
    return original_language_detox

def detox_multi_return(model, input: str, language: str):
    language_mapping = {
        'eng_Latn': 'en',  # English
        'rus_Cyrl': 'ru',  # Russian
        'ukr_Cyrl': 'uk',  # Ukrainian
        'deu_Latn': 'de',  # German
        'spa_Latn': 'es',  # Spanish
        'amh_Ethi': 'am',  # Amharic
        'zho_Hans': 'zh-cn',  # Simplified Chinese
        'arb_Arab': 'ar',  # Arabic
        'hin_Deva': 'hi'   # Hindi
    }
    language_short=language_mapping[language]
    translator=Translator()
    translated=translator.translate(input,src=language_short,dest='en').text
    detox=generate(model,translated)
    original_language_detox = translator.translate(detox,src='en',dest=language_short).text
    return (translated, detox, original_language_detox)

def calculate_mean(data, rouge_key):
    values = np.array([[entry[rouge_key]['r'], entry[rouge_key]['p'], entry[rouge_key]['f']] for entry in data])
    return values.mean(axis=0)

def loss_fn(outputs, targets,mask):
    a=nn.CrossEntropyLoss(reduction='none')(outputs,targets)
    mask=mask.float()
    b=torch.tensordot(a,mask)
    return b/torch.sum(mask)

def jaccard_similarity(text1, text2):
    """
    Computes the Jaccard similarity between two texts.
    """
    set1 = set(text1.split())  # Tokenize into words
    set2 = set(text2.split())
    intersection = len(set1 & set2)
    union = len(set1 | set2)
    return intersection / union if union != 0 else 0

# Loading and preparing Data

In [3]:
all_languages = pd.read_csv('total_dataset_translated.csv',sep='\t')

In [4]:
all_languages

,toxic_sentence_not_translated,neutral_sentence_not_translated,toxic_sentence_translated,neutral_sentence_translated,original_language
0,"then all of a sudden i see her , shes now got ...","All of a sudden i see her, she is all grown up.","then all of a sudden i see her , shes now got ...","All of a sudden i see her, she is all grown up.",eng_Latn
1,My page should be protected first so that wort...,My page should be protected first so that unpl...,My page should be protected first so that wort...,My page should be protected first so that unpl...,eng_Latn
2,You made a mistake you ass.,You made a mistake.,You made a mistake you ass.,You made a mistake.,eng_Latn
3,"you know more than these idiots , stay the cou...","you know more than these people , stay the cou...","you know more than these idiots , stay the cou...","you know more than these people , stay the cou...",eng_Latn
4,"piss me off , fuckin jerk , get on my nerves .",get on my nerves,"piss me off , fuckin jerk , get on my nerves .",get on my nerves,eng_Latn
...,...,...,...,...,...
23339,when they do shit like this .,when they do stuff like this,when they do shit like this .,when they do stuff like this,eng_Latn
23340,"but if saying "" fuck that group "" is much more...","but if saying"" that group is bad"" is much more...","but if saying "" fuck that group "" is much more...","but if saying"" that group is bad"" is much more...",eng_Latn
23341,"it hurts how judgemental assholes view them , ...",It hurts how judgemental that people view them...,"it hurts how judgemental assholes view them , ...",It hurts how judgemental that people view them...,eng_Latn
23342,shit we probably literally blow that up in a w...,We probably litteralky blow that up in a week.,shit we probably literally blow that up in a w...,We probably litteralky blow that up in a week.,eng_Latn


In [5]:
X_train,X_test,y_train,y_test = train_test_split(all_languages['toxic_sentence_translated'],all_languages['neutral_sentence_translated'],train_size=0.8,random_state=2,stratify=all_languages['original_language']) #Estratificado por idioma para manter la distribución de clases en ambos conjuntos

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, dataset,targets,max_len=50):
        self.dataset = dataset
        self.targets=targets
        self.tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
        self.max_len = max_len

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, index):

        phrase = self.dataset.iloc[index]
        target = self.targets.iloc[index]

        phrase = " ".join(phrase.split())
        inputs = self.tokenizer.encode_plus(
                phrase,
                None,
                add_special_tokens=True,
                max_length=self.max_len,
                pad_to_max_length=True,
                return_token_type_ids=True,
                truncation=True
            )
        target = " ".join(target.split())
        outputs = self.tokenizer.encode_plus(
                target,
                None,
                add_special_tokens=True,
                max_length=self.max_len,
                pad_to_max_length=True,
                return_token_type_ids=True,
                truncation=True
            )
        
        encoder_ids = inputs['input_ids']
        encoder_mask = inputs['attention_mask']
        decoder_ids = outputs['input_ids'][:-1]
        decoder_mask = outputs['attention_mask'][:-1]
        decoder_outs = outputs['input_ids'][1:]
        decoder_outs_mask = outputs['attention_mask'][1:]

        return {
            'encoder_ids': torch.tensor(encoder_ids, dtype=torch.long),
            'encoder_mask': torch.tensor(encoder_mask, dtype=torch.long),
            'decoder_input_ids': torch.tensor(decoder_ids, dtype=torch.long),
            'decoder_input_mask': torch.tensor(decoder_mask, dtype=torch.long),
            'targets': torch.tensor(decoder_outs, dtype=torch.float),
            'target_mask': torch.tensor(decoder_outs_mask, dtype=torch.float),
            'no_tokenize_target':target
        }

train_dataset = CustomDataset(X_train,y_train)
test_dataset = CustomDataset(X_test,y_test)
train_dataloader = DataLoader(train_dataset,batch_size=1,shuffle=True)
test_dataloader = DataLoader(test_dataset,batch_size=1,shuffle=True)

# Train the model

In [18]:
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
model: T5ForConditionalGeneration = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")

# Tell the T5 model which task has to carry out
model.config.task_specific_params={
    "neutralization": {
      "early_stopping": True,
      "length_penalty": 2.0,
      "max_length": 100,
      "min_length": 50,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
      "prefix": "neutralize: "
    }
}
model.to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=2e-5)

In [19]:
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Total de parámetros: {total_params:,}")
print(f"Parámetros entrenables: {trainable_params:,}")
print(f"Parámetros congelados: {total_params - trainable_params:,}")

Total de parámetros: 247,577,856
Parámetros entrenables: 247,577,856
Parámetros congelados: 0


In [ ]:
def train(model,optimizer,epochs,train_dataloader,test_dataloader):
    checkpoint_path = "t5_checkpoint.pth"
    for epoch in range(epochs):
        losses=[]
        print('-'*100)
        print(f'Epoch number: {epoch}.')
        model.train()
        time1=time.time()
        print('Training...')
        for data in tqdm(train_dataloader, desc=f"Training Epoch {epoch + 1}/{epochs}"):
            targets=data['targets'].to(device,dtype=torch.long)
            target_mask=data['target_mask'].to(device,dtype=torch.long)
            encoder_ids=data['encoder_ids'].to(device,dtype=torch.long)
            encoder_mask=data['encoder_mask'].to(device,dtype=torch.long)
            decoder_input_ids=data['decoder_input_ids'].to(device,dtype=torch.long)
            decoder_input_mask=data['decoder_input_mask'].to(device,dtype=torch.long)
            output=model.forward(encoder_ids,encoder_mask,decoder_input_ids,decoder_input_mask)
            del encoder_ids,encoder_mask,decoder_input_ids,decoder_input_mask
            output=output.logits
            output=torch.transpose(output,2,1)
            optimizer.zero_grad()
            loss=loss_fn(output,targets,target_mask)
            del targets,target_mask,output
            loss.backward()
            optimizer.step()
        all_losses=[]
        model.eval()
        print('Testing...')
        for test in tqdm(test_dataloader, desc=f"Testing Epoch {epoch + 1}/{epochs}"):
            targets=test['targets'].to(device,dtype=torch.long)
            target_mask=test['target_mask'].to(device,dtype=torch.long)
            encoder_ids=test['encoder_ids'].to(device,dtype=torch.long)
            encoder_mask=test['encoder_mask'].to(device,dtype=torch.long)
            decoder_input_ids=test['decoder_input_ids'].to(device,dtype=torch.long)
            decoder_input_mask=test['decoder_input_mask'].to(device,dtype=torch.long)
            
            output=model.forward(encoder_ids,encoder_mask,decoder_input_ids,decoder_input_mask)
            
            del encoder_ids,encoder_mask,decoder_input_ids,decoder_input_mask
            output=output.logits
            output=torch.transpose(output,2,1)
            loss=loss_fn(output,targets,target_mask)
            del output
            all_losses.append(loss.item())
            # phrases=test['no_tokenize_target']
            # predicted_target=[generate(model,target) for target in phrases]
            # out=compute_rouge(phrases,predicted_target)
            # r_1 = calculate_mean(out, 'rouge-1')
            # r_2 = calculate_mean(out, 'rouge-2')
            # r_l = calculate_mean(out, 'rouge-l')
            del targets,target_mask,
        loss_epoch=np.array(all_losses).mean()
        print(f'Loss in test set:{loss}.')
        # print(f'Rouge-1: \'r\': {r_1[0]}, \'p\': {r_1[1]}, \'f\': {r_1[2]}')
        # print(f'Rouge-2: \'r\': {r_2[0]}, \'p\': {r_2[1]}, \'f\': {r_2[2]}')
        # print(f'Rouge-l: \'r\': {r_l[0]}, \'p\': {r_l[1]}, \'f\': {r_l[2]}')
        losses.append(loss_epoch)
        print(f'Time: {time.time()-time1}')
        checkpoint = {
            "epoch": epoch,
            "model_state_dict": model.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
            "loss": losses,
        }
        torch.save(checkpoint, checkpoint_path)
        print(f"Checkpoint saved at {checkpoint_path}")

In [10]:
train(model,optimizer,10,train_dataloader,test_dataloader)

----------------------------------------------------------------------------------------------------
Epoch number: 0.
Training...


Training Epoch 1/10: 100%|██████████| 584/584 [02:58<00:00,  3.27it/s]


Testing...


Testing Epoch 1/10: 100%|██████████| 146/146 [00:17<00:00,  8.36it/s]


Loss in test set:1.1723066568374634.
Time: 196.3087465763092
Checkpoint saved at t5_checkpoint.pth
----------------------------------------------------------------------------------------------------
Epoch number: 1.
Training...


...

Loss in test set:0.6574808359146118.
Time: 196.76171827316284
Checkpoint saved at t5_checkpoint.pth
----------------------------------------------------------------------------------------------------
Epoch number: 9.
Training...


Training Epoch 10/10: 100%|██████████| 584/584 [02:59<00:00,  3.25it/s]


Testing...


Testing Epoch 10/10: 100%|██████████| 146/146 [00:16<00:00,  8.61it/s]


Loss in test set:0.8399816155433655.
Time: 196.5332260131836
Checkpoint saved at t5_checkpoint.pth


# Loading saved checkpoint

In [3]:
checkpoint_path = "t5_big.pth"
# checkpoint_path = "t5_checkpoint.pth"
checkpoint=torch.load(checkpoint_path)

device = "cuda" if torch.cuda.is_available()  else "cpu"
device

'cuda'

In [4]:
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
model:T5ForConditionalGeneration = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base", device_map=device)

# Tell the T5 model which task has to carry out
model.config.task_specific_params={
  "neutralization": {
    "early_stopping": True,
    "length_penalty": 2.0,
    "max_length": 100,
    "min_length": 50,
    "no_repeat_ngram_size": 3,
    "num_beams": 4,
    "prefix": "neutralize: "
  }
}
model.load_state_dict(checkpoint["model_state_dict"])
model.to(device)
optimizer = torch.optim.Adam(params =  model.parameters())
optimizer.load_state_dict(checkpoint["optimizer_state_dict"])

# Testing on custom english phrases (detox model)

In [5]:
def detoxify_english_example(model: T5ForConditionalGeneration, toxic_sentence: str) -> None:
    detox_sentence=generate(model,toxic_sentence)
    out=compute_rouge(toxic_sentence,detox_sentence)
    print(f'Toxic sentence: {toxic_sentence}')
    print(f'Detox sentence: {detox_sentence}')
    r_1 = calculate_mean(out, 'rouge-1')
    r_2 = calculate_mean(out, 'rouge-2')
    r_l = calculate_mean(out, 'rouge-l')
    print(f'Rouge-1: \'r\': {r_1[0]:.2f}, \'p\': {r_1[1]:.3f}, \'f\': {r_1[2]:.3f}')
    print(f'Rouge-2: \'r\': {r_2[0]:.2f}, \'p\': {r_2[1]:.3f}, \'f\': {r_2[2]:.3f}')
    print(f'Rouge-l: \'r\': {r_l[0]:.2f}, \'p\': {r_l[1]:.3f}, \'f\': {r_l[2]:.3f}')

In [6]:
# Sentence with a mixture of a bad word well used (meaning) and that same word used as an insult.
sentence='Stupid people make things wrong. You are stupid'
detoxify_english_example(model, sentence)

Toxic sentence: Stupid people make things wrong. You are stupid
Detox sentence: Stupid people make things wrong
Rouge-1: 'r': 1.00, 'p': 0.625, 'f': 0.769
Rouge-2: 'r': 1.00, 'p': 0.571, 'f': 0.727
Rouge-l: 'r': 1.00, 'p': 0.625, 'f': 0.769


In [7]:
# Sentence with a swear and a characterisc of a person
sentence='My bloody brother is a sore loser'
detoxify_english_example(model, sentence)

Toxic sentence: My bloody brother is a sore loser
Detox sentence: My brother is a loser
Rouge-1: 'r': 1.00, 'p': 0.714, 'f': 0.833
Rouge-2: 'r': 0.50, 'p': 0.333, 'f': 0.400
Rouge-l: 'r': 1.00, 'p': 0.714, 'f': 0.833


In [8]:
# Non-toxic sentence
sentence='That very person has such a lovely soul'
detoxify_english_example(model, sentence)

Toxic sentence: That very person has such a lovely soul
Detox sentence: That very person has such a lovely soul
Rouge-1: 'r': 1.00, 'p': 1.000, 'f': 1.000
Rouge-2: 'r': 1.00, 'p': 1.000, 'f': 1.000
Rouge-l: 'r': 1.00, 'p': 1.000, 'f': 1.000


In [9]:
# Full toxic sentence
sentence='I hate you, you are such a dick head, i really don´t want to play you anymore because you are a completely asshole'
detoxify_english_example(model, sentence)

Toxic sentence: I hate you, you are such a dick head, i really don´t want to play you anymore because you are a completely asshole
Detox sentence: I hate you, i really don t want to play you anymore because you are completely unsuitable
Rouge-1: 'r': 0.81, 'p': 0.684, 'f': 0.743
Rouge-2: 'r': 0.62, 'p': 0.476, 'f': 0.541
Rouge-l: 'r': 0.81, 'p': 0.684, 'f': 0.743


# Testing toxic sentences in different languages

In [14]:
def detoxify_multilanguage_example(model: T5ForConditionalGeneration, toxic_sentence: str, language: str) -> None:
    toxic_translated, detox_sentence, detox_detranslated=detox_multi_return(model,sentence,language)
    out=compute_rouge(sentence,detox_detranslated)
    print(f'Toxic sentence: {toxic_sentence}')
    print(f'Toxic translated sentence: {toxic_translated}')
    print(f'Detox sentence: {detox_sentence}')
    print(f'Detox detranslated sentence: {detox_detranslated}')
    r_1 = calculate_mean(out, 'rouge-1')
    r_2 = calculate_mean(out, 'rouge-2')
    r_l = calculate_mean(out, 'rouge-l')
    print(f'Rouge-1: \'r\': {r_1[0]:.2f}, \'p\': {r_1[1]:.3f}, \'f\': {r_1[2]:.3f}')
    print(f'Rouge-2: \'r\': {r_2[0]:.2f}, \'p\': {r_2[1]:.3f}, \'f\': {r_2[2]:.3f}')
    print(f'Rouge-l: \'r\': {r_l[0]:.2f}, \'p\': {r_l[1]:.3f}, \'f\': {r_l[2]:.3f}')

In [15]:
sentence='Eres muy tonto, que te crees listo y eres estupido.'
language='spa_Latn'
detoxify_multilanguage_example(model, sentence, language)

Toxic sentence: Eres muy tonto, que te crees listo y eres estupido.
Toxic translated sentence: You are very stupid, you think you are smart and you are stupid.
Detox sentence: You think you are smart and you are not smart.
Detox detranslated sentence: Crees que eres inteligente y no lo eres.
Rouge-1: 'r': 0.43, 'p': 0.300, 'f': 0.353
Rouge-2: 'r': 0.00, 'p': 0.000, 'f': 0.000
Rouge-l: 'r': 0.43, 'p': 0.300, 'f': 0.353


In [16]:
sentence='Du bist ein komplettes Arschloch, du bist ein beschissener Arschlecker.'
language='deu_Latn'
detoxify_multilanguage_example(model, sentence, language)

Toxic sentence: Du bist ein komplettes Arschloch, du bist ein beschissener Arschlecker.
Toxic translated sentence: You're a complete asshole, you're a fucking asslicker.
Detox sentence: You're a complete idiot, you're a bad guy.
Detox detranslated sentence: Du bist ein Vollidiot, du bist ein Bösewicht.
Rouge-1: 'r': 0.67, 'p': 0.500, 'f': 0.571
Rouge-2: 'r': 0.50, 'p': 0.375, 'f': 0.429
Rouge-l: 'r': 0.67, 'p': 0.500, 'f': 0.571


# Testing on Jaccard-Similarity

In [13]:
sentence='Eres una persona muy estúpida'
language='spa_Latn'
detox_sentence=detox(model,sentence,language)
print(f'Toxic sentence: {sentence}')
print(f'Detox sentence: {detox_sentence}')
jaccard_similarity(sentence,detox_sentence)

Toxic sentence: Eres una persona muy estúpida
Detox sentence: eres una muy mala persona


0.42857142857142855

In [ ]:
languages=[
    'eng_Latn',
    'rus_Cyrl',
    'ukr_Cyrl',
    'deu_Latn',
    'spa_Latn',
    'amh_Ethi',
    'zho_Hans',
    'arb_Arab',
    'hin_Deva'
]
all_similarities=[]
for language in languages:
    similarity=[]
    dataframe=all_languages.iloc[np.squeeze(np.argwhere(np.array(all_languages['original_language'])=='spa_Latn'),axis=-1)]
    _,X_test,_,y_test = train_test_split(dataframe['toxic_sentence_not_translated'],dataframe['neutral_sentence_not_translated'],train_size=0.8,random_state=2) #Estratificado por idioma para manter la distribución de clases en ambos conjuntos
    for sample in tqdm(X_test,desc=f"Testing on language {language}"):
        detox_sentence=detox(model,sample,language)
        sim= jaccard_similarity(sample,detox_sentence)
        similarity.append(sim)
    similarity=np.array(similarity)
    all_similarities.append(np.mean(similarity))
    print(f'J-Score in {language}: {np.mean(similarity)}')
all_similarities=np.array(all_similarities)

Testing on language eng_Latn: 100%|██████████| 80/80 [02:59<00:00,  2.25s/it]


J-Score in eng_Latn: 0.6654769663222726


Testing on language rus_Cyrl: 100%|██████████| 80/80 [02:09<00:00,  1.62s/it]


J-Score in rus_Cyrl: 0.22380855807988914


Testing on language ukr_Cyrl: 100%|██████████| 80/80 [02:03<00:00,  1.55s/it]


...

J-Score in zho_Hans: 0.1341261569786186


Testing on language arb_Arab: 100%|██████████| 80/80 [01:50<00:00,  1.38s/it]


J-Score in arb_Arab: 0.03924499549187455


Testing on language hin_Deva: 100%|██████████| 80/80 [02:07<00:00,  1.59s/it]

J-Score in hin_Deva: 0.0055321336735810415


In [11]:
print(f'Mean J-Score: {np.mean(all_similarities)}')

Mean J-Score: 0.2937465278894505


# Comparación de Resultados con otros presentados en la tarea

La métrica usada es J-Score. Se han obtenido los siguientes resultados:
- Inglés: J-Score: 0.6654 --> Top 1
- Ruso: J-Score: 0.223 --> Top 30
- Ucraniano J-Score: 0.539 --> Top 11
- Alemán J-Score: 0.2507 ---> Top 29
- Español: J-Score: 0.439 ---> Top 11
- Amárico. J-Score: 0.346 ---> Top 3
- Chino. J-Score: 0.134 ---> Top 13
- Árabe. J-Score: 0.039 ---> Top 35
- Hindú. J-Score: 0.0055 ---> Top 35
- J-Score medio de todos los idiomas: 0.293 ---> Top 25
